In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import random

# import autograd.numpy as np  # Thinly-wrapped numpy
# from autograd import grad    # The only autograd function you may ever need

### Linear Policy Graph

In [2]:
class LinearPolicyGraph: 
    def __init__(self, stages, random_vars):
        self.stages = list(range(1,stages+1))
        self.random_vars = random_vars
        self.nodes = list(range(1,stages+1))

    def sample_path(self):
        intra = []
        for i in self.stages:
            intra.append(random.choice(list(random_vars.keys())))
        return intra



In [3]:
stages = 3

intra_probs = {1:1/3, 2:1/3, 3:1/3}
random_vars = {1:{'inflow':0,'fuel':1}, 2: {'inflow':50,'fuel':1}, 3: {'inflow':100,'fuel':1}}

fuel_stage_cost = [50,100,150]

pg = LinearPolicyGraph(stages, random_vars)


In [4]:
def subproblem_builder():
    nodes = pg.nodes
    subproblems = {}
    for i in nodes:
                # placing this outside of the loop an inputing m as a parameter ends up using the same object for all subproblems
                # putting it this way makes sure that each subproblem is a different object, this is a work around just for now.
                m = ConcreteModel()

                #State Variables
                m.volume = Var(within=NonNegativeReals, bounds = (0,200))
                m.volume_in = Var(within=NonNegativeReals, bounds = (0,200))
                m.cost_to_go = Var(within=NonNegativeReals, initialize=0)
                
                m.volume_x = Param(initialize=0, mutable=True)

                #Local Variables
                m.thermal_generation = Var(within=NonNegativeReals)
                m.hydro_generation = Var(within=NonNegativeReals)
                m.hydro_spill = Var(within=NonNegativeReals)

                #Intrastage Random Variables
                m.inflow = Param(initialize=0, mutable=True)

                #Interstage Random Variables 
                m.fuelcost = Param(initialize=0, mutable=True)
                m.fuelmulti = Param(initialize=0, mutable=True)
                
                m.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

                m.obj = Objective(expr = m.fuelcost*m.fuelmulti * m.thermal_generation + m.cost_to_go, sense=minimize)

                m.con1 = Constraint(expr = m.volume == m.volume_in - m.hydro_generation - m.hydro_spill + m.inflow)
                m.con2 = Constraint(expr = m.hydro_generation + m.thermal_generation == 150)
                m.con3 = Constraint(expr = m.volume_in == m.volume_x)
                m.cuts = ConstraintList()

                subproblems[i] = m
    return subproblems

sb = subproblem_builder()
opt = SolverFactory('gurobi')


import sys, os

# Function that disables printing
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Function that enables printing
def enablePrint():
    sys.stdout = sys.__stdout__


In [5]:
forward_pass_info = {}

def forward_pass(sb, initial_states,forward_pass_info, iteration, intra, **kwargs):
        opt = SolverFactory('gurobi')

        print_log = kwargs.get("detailed_log", False)
        # replications = kwargs.get("replications", None)

        if print_log == False:
            blockPrint()
            
        print("| Forward Pass")
    
        # print("Sampling Paths\n")
        # print("sampled noise", intra)
        # print("Setting Model Parameters")
        for i in sb:        
            sb[i].fuelmulti = random_vars[intra[i-1]]['fuel']
            sb[i].inflow   = random_vars[intra[i-1]]['inflow']
            sb[i].fuelcost = fuel_stage_cost[i-1]
            # print("subproblem {} : fuel multiplier = {}, inflow = {}, fuel cost = {}".format(i, sb[i].fuelmulti(), sb[i].inflow(), sb[i].fuelcost()))
       
        node = 1

        print("| | Visiting node ", node)
    
        # print("Solving first stage problem with initial state variables with initials {}".format(initial_states))
        sb[node].volume_x = initial_states
        results = opt.solve(sb[node])
        volume_out = sb[node].volume.value
        # print("volume out {}".format(volume_out))
        # print("objective value", value(sb[node].obj))

        forward_pass_info[iteration,node] = {
                                   'inflow': sb[node].inflow.value,
                                   'fuel_cost': sb[node].fuelcost.value,
                                   'fuel_multi': sb[node].fuelmulti.value,
                                   'volume_in': sb[node].volume_in.value, 
                                   'volume_out': sb[node].volume.value,
                                   'cost_to_go': sb[node].cost_to_go.value,
                                   'stage_cost': value(sb[node].obj) - sb[node].cost_to_go.value,
                                   'obj_value' : value(sb[node].obj)}

        print("| | | w = ", sb[node].inflow.value)
        print("| | | x = ", sb[node].volume_in.value)
        print("| | | x' = ", sb[node].volume.value)
        print("| | | C(x,u,w)' = ", value(sb[node].obj) - sb[node].cost_to_go.value)


        node = 2

        print("| | Visiting node ", node)

        # print("setting volume in to {} for second subproblem to volume out of first and solving".format(volume_out))
        sb[node].volume_x = volume_out
        results = opt.solve(sb[node])
        volume_out = sb[node].volume.value
        # print("volume out {}".format(volume_out))
        # print("objective value", value(sb[node].obj))

        forward_pass_info[iteration,node] = {
                                   'inflow': sb[node].inflow.value,
                                   'fuel_cost': sb[node].fuelcost.value,
                                   'fuel_multi': sb[node].fuelmulti.value,
                                   'volume_in': sb[node].volume_in.value, 
                                   'volume_out': sb[node].volume.value,
                                   'cost_to_go': sb[node].cost_to_go.value,
                                   'stage_cost': value(sb[node].obj) - sb[node].cost_to_go.value,
                                   'obj_value' : value(sb[node].obj)}

        print("| | | w = ", sb[node].inflow.value)
        print("| | | x = ", sb[node].volume_in.value)
        print("| | | x' = ", sb[node].volume.value)
        print("| | | C(x,u,w)' = ", value(sb[node].obj) - sb[node].cost_to_go.value)


        node = 3

        print("| | Visiting node ", node)

        # print("setting volume in for third subproblem to volume out of first and solving")
        sb[3].volume_x = volume_out
        results = opt.solve(sb[3])
        volume_out = sb[3].volume.value
        # print("volume out {}".format(volume_out))
        # print("objective value", value(sb[3].obj))

        forward_pass_info[iteration,node] = {
                                   'inflow': sb[node].inflow.value,
                                   'fuel_cost': sb[node].fuelcost.value,
                                   'fuel_multi': sb[node].fuelmulti.value,
                                   'volume_in': sb[node].volume_in.value, 
                                   'volume_out': sb[node].volume.value,
                                   'cost_to_go': sb[node].cost_to_go.value,
                                   'stage_cost': value(sb[node].obj) - sb[node].cost_to_go.value,
                                   'obj_value' : value(sb[node].obj)}

        print("| | | w = ", sb[node].inflow.value)
        print("| | | x = ", sb[node].volume_in.value)
        print("| | | x' = ", sb[node].volume.value)
        print("| | | C(x,u,w)' = ", value(sb[node].obj) - sb[node].cost_to_go.value)

        enablePrint()

        # if replications != None:
        #     replication_results = {}
        #     replication_results = forward_simulations(replications, iteration, forward_pass_info, **kwargs)
            
            
        return forward_pass_info
     
#       print("solving subproblem {}, sample noise {}".format(i[1],intra[i[0]]))



In [6]:
# This has to start from the second stage
Pij = 1 # in a linear graph transition probability is always 1 because there is only one possible node at every stage

backward_pass_info = {}

def backward_pass(backward_pass_info, iteration, **kwargs): 
    print_log = kwargs.get("detailed_log", False)

    if print_log == False:
        blockPrint()
    
    print("| Backward Pass")

    opt = SolverFactory('gurobi')
    V = {}
    dxdV = {}

    node = 3

    print("| | Visiting node ", node)
    print("| | | skipping node because the cost-to-go is 0")

    #Solve the last stage problem for all scenario samples
    sb[node].cost_to_go.fix(0)
    # print("solving for every noise element at node 2")
    for i in random_vars:
        sb[node].inflow = random_vars[i]['inflow']
        results = opt.solve(sb[node])
        # print("objective value", value(sb[3].obj))
        # print("dVdx' = ", sb[3].dual[sb[3].con3])
        
        V[i] = value(sb[node].obj)
        dxdV[i] = sb[node].dual[sb[node].con3]
    # Add cut to second stage problem 
    backward_pass_info[iteration,node] = {
                               'inflow': sb[node].inflow.value,
                               'fuel_cost': sb[node].fuelcost.value,
                               'fuel_multi': sb[node].fuelmulti.value,
                               'volume_in': sb[node].volume_in.value, 
                               'volume_out': sb[node].volume.value,
                               'cost_to_go': sb[node].cost_to_go.value,
                               'obj_value' : value(sb[node].obj),
                                'V':V,
                                'dxdV':dxdV}
    

    node = 2

    # print("adding cuts to nodes 2")
    
    sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij*sum(intra_probs[p]*(V[p] + (sb[node].volume-forward_pass_info[iteration,node]['volume_out'])*dxdV[p]) for p in random_vars))
    # print(sum(intra_probs[p]*(V[p]) for p in random_vars), sum(intra_probs[p]*dxdV[p] for p in random_vars))
    # print(V, dxdV)


    print("| | Visiting node ", node)
    for i in random_vars:
        print("| | | Solving ϕ = ", random_vars[i]['inflow'])
        print("| | | | V = ", V[i])
        print("| | | | dVdx' =", dxdV[i])
        
    print("| | Adding cut :  ")
    
    # print("solving for every noise element at node 2")
    for i in random_vars:
        sb[2].inflow = random_vars[i]['inflow']
        results = opt.solve(sb[node])
        # print("objective value", value(sb[2].obj))
        # print("dVdx' = ", sb[2].dual[sb[2].con3])
        
        V[i] = value(sb[node].obj)
        dxdV[i] = sb[node].dual[sb[node].con3]

    backward_pass_info[iteration,node] = {
                               'inflow': sb[node].inflow.value,
                               'fuel_cost': sb[node].fuelcost.value,
                               'fuel_multi': sb[node].fuelmulti.value,
                               'volume_in': sb[node].volume_in.value, 
                               'volume_out': sb[node].volume.value,
                               'cost_to_go': sb[node].cost_to_go.value,
                               'obj_value' : value(sb[node].obj),
                                'V':V,
                                'dxdV':dxdV}
    

    node = 1

    # print("adding cuts to node 1")
    sb[node].cuts.add(expr = sb[node].cost_to_go >= Pij*sum(intra_probs[p]*(V[p] + (sb[node].volume-forward_pass_info[iteration,node]['volume_out'])*dxdV[p]) for p in random_vars))
    # print(sum(intra_probs[p]*(V[p]) for p in random_vars), sum(intra_probs[p]*dxdV[p] for p in random_vars))
    # print(V,dxdV)
#     sb[1].cuts.add(expr = 133.33333333333331*sb[1].volume + sb[1].cost_to_go >= 22500)


    print("| | Visiting node ", node)
    for i in random_vars:
        print("| | | Solving ϕ = ", random_vars[i]['inflow'])
        print("| | | | V = ", V[i])
        print("| | | | dVdx' =", dxdV[i])

    print("| | Adding cut :  ")

    # print("solving for every noise element at node 1 to compute the lower bound")
    for i in random_vars:
        sb[node].inflow = random_vars[i]['inflow']
        results = opt.solve(sb[node])
        V[i] = value(sb[node].obj)
    lower_bound = sum(intra_probs[p]*V[p] for p in random_vars)
    
    enablePrint()
    
    print("| Finished iteration ", iteration)
    print("| | lower bound = ", lower_bound)

    backward_pass_info[iteration,node] = {
                           'inflow': sb[node].inflow.value,
                           'fuel_cost': sb[node].fuelcost.value,
                           'fuel_multi': sb[node].fuelmulti.value,
                           'volume_in': sb[node].volume_in.value, 
                           'volume_out': sb[node].volume.value,
                           'cost_to_go': sb[node].cost_to_go.value,
                           'obj_value' : value(sb[node].obj),
                            'V':V,
                            'dxdV':dxdV,
                            'lower_bound': lower_bound}

    
    return backward_pass_info

In [7]:
def forward_simulations(rep_num, iteration, forward_pass_info, **kwargs):
    rep_key = 'simulations of {}'.format(iteration)
    replications = list(range(1,rep_num + 1))
    rep_results = {}
    for i in replications:
        intra = pg.sample_path()  
        forward_pass(sb,200, forward_pass_info, rep_key, intra, detailed_log=False)
        rep_results[rep_key,i] = sum([forward_pass_info[rep_key,1]['stage_cost'],forward_pass_info[rep_key,2]['stage_cost'],forward_pass_info[rep_key,3]['stage_cost']])


        z_list = list(rep_results.values())

    print("| Simulation CI: ", np.mean(z_list), u"\u00B1", (np.std(z_list)*1.96) / np.sqrt(rep_num))

    return rep_results

In [8]:
iter_num = 50
sb = subproblem_builder()
for iteration in range(1,iter_num):
    print("Starting Iteration ", iteration)
    intra = pg.sample_path() 
    forward_pass(sb,200, forward_pass_info, iteration, intra, detailed_log=False)
    backward_pass(backward_pass_info,iteration, detailed_log=False)


simulation_results = forward_simulations(50,iteration, forward_pass_info)

Starting Iteration  1
| Finished iteration  1
| | lower bound =  2500.0
Starting Iteration  2
| Finished iteration  2
| | lower bound =  7500.0
Starting Iteration  3
| Finished iteration  3
| | lower bound =  7833.333333333334
Starting Iteration  4
| Finished iteration  4
| | lower bound =  8333.333333333332
Starting Iteration  5
| Finished iteration  5
| | lower bound =  8333.333333333332
Starting Iteration  6
| Finished iteration  6
| | lower bound =  8333.333333333332
Starting Iteration  7
| Finished iteration  7
| | lower bound =  8333.333333333332
Starting Iteration  8
| Finished iteration  8
| | lower bound =  8333.333333333332
Starting Iteration  9
| Finished iteration  9
| | lower bound =  8333.333333333332
Starting Iteration  10
| Finished iteration  10
| | lower bound =  8333.333333333332
Starting Iteration  11
| Finished iteration  11
| | lower bound =  8333.333333333332
Starting Iteration  12
| Finished iteration  12
| | lower bound =  8333.333333333332
Starting Iteration  

In [ ]:
simulation_results